# Extracting Data via the Liberator API

In this notebook, we demonstronstrate how to use CloudQuant's Liberator API to download several datasets. To use the Liberator API, you'll need an access key. You can reach out to [CloudQuant](https://www.cloudquant.com/) to find out more about the API and how to obtain access.

<center><img src="./images/AtotiCommunity.png" alt="Atoti Community" style="height: 100px;" hspace="50"></a></div><img src="./images/CloudQuantPNGLogo.png" alt="CloudQuant" style="height: 100px;" hspace="50"></a></div></center>

In [1]:
import liberator
import pandas as pd
from utils.mapping import nasdaq_100

## Daily Bars Data

In [2]:
daily_bars_df = liberator.get_dataframe(
    liberator.query(
        name = 'daily_bars',
        symbols = nasdaq_100,
        as_of = '2023-06-30',
        back_to = '2023-01-02'
    ),
)

In [3]:
daily_bars_df = daily_bars_df.drop(columns=['_seq', '_dsname', 'msg_len'])
daily_bars_df = daily_bars_df.astype({'muts': 'int64','length': 'int64','count': 'int32', 'bidvol': 'int32', 'askvol': 'int32'})

dates_df = daily_bars_df["timestamp"]

daily_bars_df.astype({"close":"int64"})

In [6]:
daily_bars_array_df = daily_bars_df[["timestamp","symbol","close"]].pivot(index="timestamp",columns="symbol",values="close")

In [7]:
daily_bars_array_df.index = pd.to_datetime(daily_bars_array_df.index)

In [9]:
def get_historical_vector(df_close):
    
    # compute returns
    df_daily_ret = df_close.diff().iloc[1:]
    df_daily_ret = df_daily_ret[(df_daily_ret == 0).sum(1) < 2]
    df_daily_ret_t = df_daily_ret.T
    df_daily_ret_t["daily_returns_vector"] = df_daily_ret_t.values.tolist()
    df_daily_ret_t = (
        df_daily_ret_t[["daily_returns_vector"]].rename_axis("symbol").reset_index()
    )

    # compute daily Rate of Returns
    df_daily_ror = df_close.resample("D").last().pct_change().iloc[1:]
    df_daily_ror = df_daily_ror[(df_daily_ror == 0).sum(1) < 2]
    df_daily_ror_t = df_daily_ror.T
    df_daily_ror_t["daily_ROR_vector"] = df_daily_ror_t.values.tolist()
    df_daily_ror_t = (
        df_daily_ror_t[["daily_ROR_vector"]].rename_axis("symbol").reset_index()
    )

    # compute monthly Rate of Returns
    df_mthly_ror = df_close.resample("M").last().pct_change().iloc[1:]
    df_mthly_ror = df_mthly_ror[(df_mthly_ror == 0).sum(1) < 2]
    df_mthly_ror_t = df_mthly_ror.T
    df_mthly_ror_t["monthly_ROR_vector"] = df_mthly_ror_t.values.tolist()
    df_mthly_ror_t = (
        df_mthly_ror_t[["monthly_ROR_vector"]].rename_axis("symbol").reset_index()
    )

    # ensure the dateline between daily ROR and the price vectors are consistent
    left, right = df_close.align(df_daily_ret, join="outer", axis=0)
    left = left.fillna(method="ffill").iloc[1:]

    # transform daily pricing
    df_price = left.T
    df_price["closing_price"] = df_price.values.tolist()
    df_price = df_price[["closing_price"]].rename_axis("symbol").reset_index()

    df_ror = pd.merge(df_daily_ret_t, df_daily_ror_t, on="symbol")
    df_ror = pd.merge(df_ror, df_mthly_ror_t, on="symbol")
    df_ror = pd.merge(df_ror, df_price, on="symbol")

    # get vector index
    daily_dates = pd.DataFrame(data={"historical_date": df_daily_ror.index.to_list()})
    daily_dates = daily_dates.rename_axis("date_index").reset_index()

    mthly_dates = pd.DataFrame(data={"historical_date": df_mthly_ror.index.to_list()})
    mthly_dates = mthly_dates.rename_axis("monthly_date_index").reset_index()

    df_dates = pd.merge(daily_dates, mthly_dates, on="historical_date", how="left")
    df_dates = df_dates.fillna(-1)
    df_dates[["date_index", "monthly_date_index"]] = df_dates[
        ["date_index", "monthly_date_index"]
    ].astype(int)
    
    df_ror["daily_returns_vector"] = [",".join(map(str, l)) for l in df_ror["daily_returns_vector"]]
    df_ror["daily_ROR_vector"] = [",".join(map(str, l)) for l in df_ror["daily_ROR_vector"]]
    df_ror["monthly_ROR_vector"] = [",".join(map(str, l)) for l in df_ror["monthly_ROR_vector"]]
    df_ror["closing_price"] = [",".join(map(str, l)) for l in df_ror["closing_price"]]

    return df_ror, df_dates

In [10]:
df_ror, df_dates = get_historical_vector(daily_bars_array_df)

In [11]:
df_ror.to_csv(
    "./data/array/daily_bars.csv",
    index = False
)

In [12]:
df_dates.to_csv("./data/array/historical_dates.csv", index = False)

## Alphaflow Data

In [14]:
cq_alphaflow_daily_df = liberator.get_dataframe(
    liberator.query(
        name = 'cq_alphaflow_dv_daily',
        symbols = nasdaq_100,
        as_of = '2023-06-30',
        back_to = '2023-01-02'
    )
)

In [15]:
cq_alphaflow_daily_df = cq_alphaflow_daily_df.astype({'muts': 'int64'})
cq_alphaflow_daily_df = cq_alphaflow_daily_df.drop(columns=['_seq'])

In [16]:
cq_alphaflow_daily_df.to_csv("./data/cq_alphaflow_daily.csv", index = False)

## ESG Data

In [18]:
gsq_measure_largecap_df = liberator.get_dataframe(
    liberator.query(
        name = 'gsq_measure_largecap',
        symbols = nasdaq_100,
        as_of = '2023-06-30',
        back_to = '2023-01-02'
    ),
)

In [20]:
gsq_measure_large_cap_df = gsq_measure_largecap_df.drop(
    columns=['_seq', 'muts']
)

# gsq_measure_large_cap_df.index = pd.to_datetime(gsq_measure_large_cap_df.index)
gsq_measure_large_cap_df["Delivery_Date"] = pd.to_datetime(gsq_measure_large_cap_df["Delivery_Date"])

In [21]:
gsq_measure_large_cap_df.to_csv("./data/gsq_measure_large_cap.csv", index = False)